## Installing transformer library and other dependences

In [1]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410

## importing all the libraries

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from transformers.keras_callbacks import PushToHubCallback

## Getting dataset from Hugging face

In [3]:
dataset = load_dataset("GonzaloA/fake_news")
dataset['train'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/24353 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8117 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8117 [00:00<?, ? examples/s]

{'Unnamed: 0': 0,
 'title': ' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)',
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the capti

## Connecting to API for Future deployment  

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
texttokenizer = AutoTokenizer.from_pretrained("avichr/heBERT_sentiment_analysis")
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to each text entry in the dataset
dataset = dataset.map(lambda example: {'text': remove_stopwords(example['text'])})

def preprocess_function(data):
    return texttokenizer(data["text"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Map:   0%|          | 0/24353 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

In [6]:
tokenized_news_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=texttokenizer, return_tensors="tf")
accuracy = evaluate.load("accuracy")

Map:   0%|          | 0/24353 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

In [7]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
id2label = {0: "Positive", 1: "Negative"}
label2id = {"Positive": 0, "Negative": 1}

In [9]:
batch_size = 1
num_epochs = 5
batches_per_epoch = len(tokenized_news_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

## model creating and training

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "avichr/heBERT_sentiment_analysis", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_news_dataset["train"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_news_dataset["test"],
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator
)

In [ ]:
model.compile(optimizer=optimizer)
push_to_hub_callback = PushToHubCallback(
    output_dir="fakenew_model",
    tokenizer=texttokenizer,
)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history = model.fit(x=tf_validation_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Poojithpoosa/fakenew_model into local empty directory.


Download file tf_model.h5:   0%|          | 7.45k/418M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/418M [00:00<?, ?B/s]

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  15/8117 [..............................] - ETA: 9:14:29 - loss: 0.8675

In [ ]:
text = "As a Wellesley graduate, I was shocked and dismayed by Hillary Clinton s commencement address. Actually, that is #FakeNews. I was neither shocked nor dismayed by Mrs. Clinton s speech (but I did graduate from Wellesley with honors!) It was exactly the kind of self-pitying, leaden and all-about-me performance that I would have expected from the entitled former Democrat candidate and now sore loser.Watch a few of Hillary s greatest hits during her commencement speech at Wellesley College:Also, it revealed this: Hillary hasn’t learned a thing. As her friend Mike Bloomberg recently told an audience of Harvard Business School grads Hillary lost the election because she had absolutely no message. She’still doesn’t.In addition, she is utterly lacking in self-awareness. She’spoke at length about what she’sees as a full-fledged assault on truth and reason, telling the graduating seniors that the future of America, indeed the future of the world, depends on brave, thoughtful people like you insisting on truth and integrity right now every day. This, from a woman who ran for our nation s highest office while distrusted by nearly two-thirds of the country, with good reason.As long as Mrs. Clinton and her loyalists blame James Comey, Vladimir Putin, sexism and the alt-right for her loss, Democrats will never rebuild their bridges to the working class Americans who elected Donald Trump.In her remarks, Hillary criticized people aka Trump followers, presumably on social media for sowing division at a time when we desperately need unity, and then proceeded to give a highly partisan and divisive speech.Did she ever consider that there might be families in the audience who voted for her opponent? That they might not enjoy her taking veiled potshots at President Trump, and that this was their daughters day too?Echoing the liberal media, Hillary went after President Trump through innuendo and clich s, talking about the dangers of fomenting fear and of alternative facts. Good thing there was no Q&A; someone might have wanted further clarification of how she came under sniper fire in Bosnia, or how a video caused the Benghazi attack or how the Monica Lewinsky scandal stemmed from a vast right-wing conspiracy and not Bill Clinton s unruly libido.Hillary trod dangerous pathways as she remarked, when people in power invent their own facts and attack those who question them, it can mark the beginning of the end of a free society. She is correct, of course, but a cursory review of her multiple dodges and lies about her mishandling of classified information and efforts to destroy evidence reveals a woman who fully earned her low trustworthiness ratings from Americans.The election is over, but Hillary cannot let it go. This is good news for Republicans. As long as Mrs. Clinton and her loyalists blame James Comey, Vladimir Putin, sexism and the alt-right for her loss, Democrats will never rebuild their bridges to the working class Americans who elected Donald Trump.Hillary Clinton missed an opportunity at Wellesley. She could have delivered a message of unity, and helped our anxious country begin to heal. Her followers need to see her putting country above politics; they need to see her as she briefly appeared at the inauguration, supporting the peaceful and legitimate transfer of power. Apparently, that is just too heavy a lift for Mrs. Clinton. She is happier nursing her wounds, and continuing to inflict them on the country.The former First Lady could also have talked about the dramatic changes taking place in our economy, and how women are thriving as entrepreneurs. She could have talked about the founders of Rent the Runway, or Sarah Blakely, who created Spanx, or Essie Weingarten, who bet her life s savings on a new idea and created a nail polish empire sold to L Oreal in 2010. These women are inspirational; they’didn’t shatter glass ceilings, they ignored them. It is true we need women in politics, and running NGOs, but we also need women making money and boosting the economy. That s where the funding for free college tuition and refugee services comes from a lesson Hillary may never have learned.The New York Times tells us that the’students cheered Mrs. Clinton thunderously. Of course they’did. They have been weaned on courses like Love and Intimacy in the Women s and Gender Studies department or Changing Gender Constructions in the Modern Middle East in the History Department. There s nothing wrong with those studies, unless they crowd out learning how the progressive policies of Bernie Sanders and Mrs. Clinton have failed in France, for instance, leading to a stagnant economy with few opportunities for progress. Or how capitalism has delivered billions from poverty, including in Communist China, where the growth miracle of recent decades was facilitated by policies that encouraged entrepreneurship and private enterprise.Hillary Clinton missed some history lessons too, apparently. She alluded to Nixon s presidency which would eventually end in disgrace with his impeachment for obstruction of justice. That isn’t true, of course; Nixon resigned from office. It was her husband Bill who was impeached. Amazing she would get that wrong, in delivering what was sure to be a much-reviewed address. Maybe a little too much of that Chardonnay, Mrs. Clinton? FOX NewsFor more visit LizPeek.com. Follow her on Twitter@LizPeek."

In [ ]:
classifier = pipeline("sentiment-analysis", model="anathan1997/fakenew_model")
classifier(text)

In [ ]:
optimizer, schedule = create_optimizer(init_lr=2e-8, num_warmup_steps=0, num_train_steps=total_train_steps)
model = TFAutoModelForSequenceClassification.from_pretrained(
    "avichr/heBERT_sentiment_analysis", num_labels=2, id2label=id2label, label2id=label2id
)
push_to_hub_callback = PushToHubCallback(
    output_dir="fakenew_model",
    tokenizer=texttokenizer,
)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history = model.fit(x=tf_validation_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)
classifier = pipeline("sentiment-analysis",model="anathan1997/fakenew_model")
classifier(text)

In [ ]:
optimizer, schedule = create_optimizer(init_lr=2e-11, num_warmup_steps=0, num_train_steps=total_train_steps)
model = TFAutoModelForSequenceClassification.from_pretrained(
    "avichr/heBERT_sentiment_analysis", num_labels=2, id2label=id2label, label2id=label2id
)
push_to_hub_callback = PushToHubCallback(
    output_dir="fakenew_model",
    tokenizer=texttokenizer,
)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history = model.fit(x=tf_validation_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)
classifier = pipeline("sentiment-analysis", model="anathan1997/fakenew_model")
classifier(text)